In [55]:
import json
import numpy as np
import pandas as pd
import math

In [108]:
path_to_data="/Users/carelchay/Desktop/School/Modules/DSA4262/Project 2/data/data.json"
path_to_labels = "/Users/carelchay/Desktop/School/Modules/DSA4262/Project 2/data/data.info"

In [3]:
# To be able to import the file
import sys
pathname="/Users/carelchay/Desktop/School/Modules/DSA4262/Project 2/DSA4262-frontasticfour/scripts"
if pathname not in sys.path:
    sys.path.append(pathname)

In [221]:
import getData as gd
import importlib
importlib.reload(gd)

<module 'getData' from '/Users/carelchay/Desktop/School/Modules/DSA4262/Project 2/DSA4262-frontasticfour/scripts/getData.py'>

In [69]:
getDat = gd.getData(path_to_data=path_to_data, path_to_labels=path_to_labels)

In [117]:
k_mers = getDat.get_unique_kmers()
labels = getDat.get_labels()
df = getDat.get_data(num_entries=10)
df = pd.merge(df, labels, how="left", left_on = ("transcript", "position"), 
              right_on = ("transcript_id", "transcript_position"))
df = df[['gene_id', 'transcript', 'position', "k-mer bases", "values", "label"]]
# df[['k-1', 'k', 'k+1']]=df.apply(lambda x : [x['k-mer bases'][i:i+5] for i in range(3)], axis=1, result_type="expand")

In [95]:
df

,gene_id,transcript,position,k-mer bases,values,label
0,ENSG00000004059,ENST00000000233,244,AAGACCA,"[[0.00299, 2.06, 125.0, 0.0177, 10.4, 122.0, 0...",0
1,ENSG00000004059,ENST00000000233,261,CAAACTG,"[[0.0126, 1.95, 111.0, 0.0125, 1.27, 108.0, 0....",0
2,ENSG00000004059,ENST00000000233,316,GAAACAG,"[[0.00432, 2.02, 104.0, 0.00299, 3.56, 99.3, 0...",0
3,ENSG00000004059,ENST00000000233,332,AGAACAT,"[[0.0134, 4.71, 132.0, 0.00447, 4.24, 98.8, 0....",0
4,ENSG00000004059,ENST00000000233,368,AGGACAA,"[[0.015, 6.97, 118.0, 0.0106, 3.04, 123.0, 0.0...",0
...,...,...,...,...,...,...
995,ENSG00000061794,ENST00000081029,1747,GTGACAA,"[[0.0027, 1.89, 102.0, 0.00782, 6.69, 116.0, 0...",0
996,ENSG00000061794,ENST00000081029,1758,AGAACTA,"[[0.0094, 5.34, 132.0, 0.00232, 3.2, 102.0, 0....",0
997,ENSG00000061794,ENST00000081029,1771,GAAACCC,"[[0.0189, 5.46, 108.0, 0.00266, 0.712, 108.0, ...",0
998,ENSG00000063177,ENST00000084795,118,AGAACCA,"[[0.00498, 10.6, 132.0, 0.0139, 5.17, 92.5, 0....",0


In [75]:
q = labels.groupby("gene_id")['label'].count().reset_index()

In [76]:
q['label'].sum()

121838

In [88]:
thresh=round(0.7*121838)

In [78]:
total = 0
dct = []
for idx, row in q.iterrows():
    total += row['label']
    dct.append(row['gene_id'])
    if total >=thresh:
        break

In [249]:
import prepData as prepD
import importlib
importlib.reload(prepD)

<module 'prepData' from '/Users/carelchay/Desktop/School/Modules/DSA4262/Project 2/DSA4262-frontasticfour/scripts/prepData.py'>

In [101]:
train_genes = prepD.splitdata(label_df=labels)

In [103]:
len(train_genes)

2602

In [104]:
labels.loc[labels['gene_id'].isin(train_genes), :]

,gene_id,transcript_id,transcript_position,label
1,ENSG00000004059,ENST00000000233,244,0
2,ENSG00000004059,ENST00000000233,261,0
3,ENSG00000004059,ENST00000000233,316,0
4,ENSG00000004059,ENST00000000233,332,0
5,ENSG00000004059,ENST00000000233,368,0
...,...,...,...,...
121733,ENSG00000092621,ENST00000641375,1523,0
121734,ENSG00000092621,ENST00000641375,1586,0
121735,ENSG00000092621,ENST00000641375,1996,0
121736,ENSG00000092621,ENST00000641375,2022,0


In [105]:
labels.shape

(121838, 4)

In [250]:
prepData = prepD.prepData(train_genes=train_genes, path_to_data=path_to_data, path_to_labels=path_to_labels)

In [251]:
prepData.df

,gene_id,transcript,position,k-mer bases,values,label
0,ENSG00000004059,ENST00000000233,244,AAGACCA,"[[0.00299, 2.06, 125.0, 0.0177, 10.4, 122.0, 0...",0
1,ENSG00000004059,ENST00000000233,261,CAAACTG,"[[0.0126, 1.95, 111.0, 0.0125, 1.27, 108.0, 0....",0
2,ENSG00000004059,ENST00000000233,316,GAAACAG,"[[0.00432, 2.02, 104.0, 0.00299, 3.56, 99.3, 0...",0
3,ENSG00000004059,ENST00000000233,332,AGAACAT,"[[0.0134, 4.71, 132.0, 0.00447, 4.24, 98.8, 0....",0
4,ENSG00000004059,ENST00000000233,368,AGGACAA,"[[0.015, 6.97, 118.0, 0.0106, 3.04, 123.0, 0.0...",0
5,ENSG00000004059,ENST00000000233,404,AGAACAC,"[[0.00444, 5.4, 131.0, 0.00664, 7.26, 102.0, 0...",0
6,ENSG00000004059,ENST00000000233,431,TGGACAG,"[[0.0102, 2.78, 118.0, 0.00697, 6.35, 119.0, 0...",0
7,ENSG00000004059,ENST00000000233,440,ATGACCG,"[[0.00432, 2.94, 97.5, 0.00531, 11.0, 120.0, 0...",0
8,ENSG00000004059,ENST00000000233,471,TGAACTC,"[[0.00266, 10.6, 125.0, 0.00681, 2.87, 104.0, ...",0
9,ENSG00000004059,ENST00000000233,539,AGGACAT,"[[0.017, 6.09, 115.0, 0.00864, 5.81, 120.0, 0....",0


In [252]:
X, Y = prepData[0]

In [253]:
X

tensor([[1.2300e-02, 6.6800e+00, 1.2600e+02, 8.5400e-03, 1.1900e+01, 1.2300e+02,
         2.3200e-03, 1.3700e+00, 7.8400e+01, 4.3000e+01, 5.8000e+01, 4.2000e+01],
        [1.2000e-02, 3.3700e+00, 1.2800e+02, 4.6500e-03, 1.5200e+01, 1.2400e+02,
         1.0700e-02, 4.8700e+00, 8.2300e+01, 4.3000e+01, 5.8000e+01, 4.2000e+01],
        [1.2000e-02, 3.4400e+00, 1.2700e+02, 1.6300e-02, 8.8000e+00, 1.2800e+02,
         5.9800e-03, 6.5800e+00, 8.3600e+01, 4.3000e+01, 5.8000e+01, 4.2000e+01],
        [9.9600e-03, 2.9600e+00, 1.3000e+02, 7.5500e-03, 1.2200e+01, 1.2300e+02,
         1.0300e-02, 5.5000e+00, 8.3100e+01, 4.3000e+01, 5.8000e+01, 4.2000e+01],
        [4.9800e-03, 2.1200e+00, 1.2800e+02, 1.1500e-02, 5.3400e+00, 1.2600e+02,
         4.9800e-03, 6.8500e+00, 8.2000e+01, 4.3000e+01, 5.8000e+01, 4.2000e+01],
        [2.5200e-02, 3.5800e+00, 1.2600e+02, 6.4400e-03, 9.3100e+00, 1.2600e+02,
         6.3100e-03, 7.6900e+00, 8.2400e+01, 4.3000e+01, 5.8000e+01, 4.2000e+01],
        [2.5900e-02, 6

In [263]:
dataloader = torch.utils.data.DataLoader(dataset= prepData, batch_size=5)

In [264]:
dataiter = iter(dataloader)
data= dataiter.next()

In [265]:
features, label = data

In [280]:
net = m6aNet(5, 20)
res = net.forward(features)

In [281]:
res

tensor([[0.4214, 0.4168, 0.4286, 0.4306, 0.4238, 0.4205, 0.4224, 0.4001, 0.4130,
         0.4278, 0.4066, 0.4280, 0.4803, 0.4108, 0.4073, 0.4251, 0.4180, 0.4295,
         0.4276, 0.4310],
        [0.4413, 0.4539, 0.4542, 0.4495, 0.4445, 0.4493, 0.4479, 0.4505, 0.4454,
         0.4568, 0.4415, 0.4458, 0.4490, 0.4467, 0.4469, 0.4544, 0.4510, 0.4551,
         0.4487, 0.4424],
        [0.4740, 0.4645, 0.4561, 0.4571, 0.4691, 0.4669, 0.4558, 0.4614, 0.4633,
         0.4590, 0.4694, 0.4735, 0.4713, 0.4571, 0.4686, 0.4659, 0.4613, 0.4636,
         0.4554, 0.4578],
        [0.4664, 0.4756, 0.4814, 0.4710, 0.4836, 0.4487, 0.4792, 0.4669, 0.4736,
         0.4738, 0.4695, 0.4732, 0.4633, 0.4850, 0.4848, 0.4680, 0.4738, 0.4713,
         0.4815, 0.4685],
        [0.4610, 0.4564, 0.4523, 0.4615, 0.4772, 0.4635, 0.4506, 0.4648, 0.4690,
         0.4604, 0.4614, 0.4561, 0.4650, 0.4529, 0.4517, 0.4658, 0.4609, 0.4640,
         0.4520, 0.4573]], grad_fn=<ReshapeAliasBackward0>)

In [162]:
import torch
import torch
import torch.nn as nn

In [279]:
class m6aNet(nn.Module):
    def __init__(self, batchsize, readsize):
        self.batchsize = batchsize
        self.readsize = readsize
        super(m6aNet, self).__init__()
        # Embedding Layer
        self.embed = nn.Embedding(66, 2)

        ## First Layer ##
        self.read_level_prob_1 = nn.Linear(15, 150)
        # First Batch Norm Layer
        self.norm_1 = nn.BatchNorm1d(num_features=150)
        # First Activation Layer
        self.activ_1=nn.ReLU()

        ## Second Layer ##
        self.read_level_prob_2 = nn.Linear(150, 32)
        # Second Activation Layer
        self.activ_2=nn.ReLU()

        ## Third Layer ##
        self.read_level_prob_3 = nn.Linear(32, 1)
        # Sigmoid Activation
        self.sig_1 = nn.Sigmoid()

    
    
    def forward(self, x):
        ### X is a numpy array of shape (batchsize, readsize=20, 12) ###
        
#         # Convert to tensor
#         x = torch.tensor(x)
        # Extract numeric features        
        numerics = x[:, :, :9]
        # # Extract Bases
        bases = x[:, :, 9:].type(torch.int64)
        # # Feed to embedding layer
        bases = self.embed(bases)

        # Reshape
        bases = bases.reshape(self.batchsize, self.readsize, 3*2)
        # Combine embedded output with numeric features
        x = torch.concat((numerics, bases), 2).type(torch.float)

        #### Feed Forward  ####

        ## First Layer ##
        x = self.read_level_prob_1(x)
        # First Batch Norm Layer
        x = x.transpose(dim0=1, dim1=2) # Need to transpose first
        x = self.norm_1(x)
        x = x.transpose(dim0=1, dim1=2) # Then transpose back
        # First Activation Layer
        x= self.activ_1(x)

        ## Second Layer ##
        x = self.read_level_prob_2(x)
        # Second Activation Layer
        x = self.activ_2(x)

        ## Third Layer ##
        x = self.read_level_prob_3(x)
        # Sigmoid Activation
        x = self.sig_1(x)
        x = x.reshape(-1, self.readsize)
        return x
        # Final Output
        r = 1 - torch.prod(1 - x, axis=1)
        return r

            
        
        
        
        
        

In [204]:
torch.from_numpy(np.array(int(df.iloc[0, -1])))

tensor(0)

In [201]:
df.dtypes

gene_id        object
transcript     object
position        int64
k-mer bases    object
values         object
label          object
dtype: object